<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/sketch/justin/SPARK_Extract_And_Clean_Social_Distancing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [1]:
# Install required dependancies
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 63kB/s 
     |████████████████████████████████| 204kB 37.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=ab7928a39659b21dd2144ca536212a0328c38cbdecf0835ae8c6bba9c068f235
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [2]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [4]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
# Chicago Counties
fips = ['17031','17037','17043','17063','17091','17089','17093','17111','17197','17097','18073','18089','18111','18127','55059']
f_string = " ('17031','17037','17043','17063','17091','17089','17093','17111','17197','17097','18073','18089','18111','18127','55059') "

path_2020 = 'drive/MyDrive/big-data-project/data/unprocessed/social-distancing/2020/'
target_2020 = 'drive/MyDrive/big-data-project/data/clean-data/ch/social/2020/'

path_2019 = 'drive/MyDrive/big-data-project/data/unprocessed/social-distancing/2019/'
target_2019 = 'drive/MyDrive/big-data-project/data/clean-data/ch/social/2019/'

In [6]:
def filter_fips(df, fips=fips):
    return df[df['cbg'].astype(str).str[:5].isin(fips)]

In [7]:
def get_fips(fname, filepath, fips, dirout):
    df = pd.read_csv(filepath, compression='gzip', dtype={'origin_census_block_group': object})
    df = df.rename(columns={'origin_census_block_group':'cbg'})
    df = filter_fips(df)
    df.to_csv(dirout)

In [11]:
!ls drive/MyDrive/big-data-project/data/unprocessed/social-distancing/

2020-01-01-social-distancing.csv.gz


In [13]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/unprocessed/social-distancing/')
y = '2020'

In [16]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/y/'*/*/*.csv.gz'))
df_soc.show()

+-------------------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|origin_census_block_group|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|media

In [18]:
df_soc.printSchema()

root
 |-- origin_census_block_group: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_time: string (n

In [24]:
# df_soc.limit(100).show()

KeyboardInterrupt: ignored

In [22]:
df_soc.limit(100).createOrReplaceTempView('T')
spark.sql('SELECT * FROM T WHERE `origin_census_block_group` IN ' + f_string).show()

KeyboardInterrupt: ignored

In [ ]:
df_soc.createOrReplaceTempView('T')
df_filter = spark.sql('SELECT * FROM T WHERE `origin_census_block_group` IN ' + f_string)
df_filter.show()